In [234]:
import canmatrix
import copy


frame = canmatrix.Frame("my_frame", size=8)

frame.add_signal(canmatrix.Signal(name = "Sample_signal", start_bit = 2, size = 13, is_little_endian=False ))
frame.add_signal(canmatrix.Signal(name = "Sample_signal2", start_bit = 17, size = 14, is_little_endian=False))
frame.add_signal(canmatrix.Signal(name = "Sample_signal3", start_bit = 35, size = 6, is_little_endian=False))
frame.add_signal(canmatrix.Signal(name = "Sample_signal4", start_bit = 49, size = 8, is_little_endian=False))


Signal(name='Sample_signal4', start_bit=49, size=8, is_little_endian=False, is_signed=True, offset=Decimal('0'), factor=Decimal('1'), unit='', receivers=[], comment=None, multiplex=None, mux_value=None, is_float=False, is_ascii=False, type_label='', enumeration=None, comments={}, attributes={}, values={}, mux_val_grp=[], muxer_for_signal=None, calc_min_for_none=True, calc_max_for_none=True, cycle_time=0, initial_value=Decimal('0'), min=Decimal('-128'), max=Decimal('127'))

In [251]:
frame = canmatrix.Frame("my_frame", size=8)

frame.add_signal(canmatrix.Signal(name = "Sample_signal", start_bit = 12, size = 12, is_little_endian=True))
frame.add_signal(canmatrix.Signal(name = "Sample_signal2", start_bit = 17, size = 9, is_little_endian=True))
frame.add_signal(canmatrix.Signal(name = "Sample_signal3", start_bit = 33, size = 5, is_little_endian=True))
frame.add_signal(canmatrix.Signal(name = "Sample_signal4", start_bit = 48, size = 9, is_little_endian=True))

cm = canmatrix.CanMatrix()
cm.add_frame(frame)
canmatrix.formats.dumpp({"":cm}, "aa.dbc")

In [246]:
def simple_show_frame(frame):
    layout = frame.get_frame_layout()
    for bit_nr, signal_list in enumerate(layout):
        if bit_nr % 8 == 0:
            print("")
        if signal_list == []:           
            print(".", end="")
        else:
#            print("#", end="")
            print(frame.signals.index(signal_list[0]), end="")

simple_show_frame(frame)
#frame.signals[-1].start_bit
#print("\n",frame.signals[-1].get_startbit(False))




........
0000....
00000000
......11
..22222.
........
33333333
.......3

In [247]:
test_frame = copy.deepcopy(frame)
#test_frame.signals.remove(test_frame.signals[1])
simple_show_frame(test_frame)
#test_frame.signals[1].get_startbit(False)



........
0000....
00000000
......11
..22222.
........
33333333
.......3

In [248]:
def compress(self, frame):
    for signal in frame.signals:
        if signal.is_little_endian:
            return self._compress_little(frame)
    gap_found = True
    while gap_found:
        gap_found = False
        layout = frame.get_frame_layout()
        free_start = None
        for bit_nr, signal_list in enumerate(layout):
            if signal_list == []:
                if free_start is None:
                    free_start = bit_nr
            else:
                if free_start is not None:
                    signal = layout[bit_nr][0] 
                    signal.start_bit = free_start
                    gap_found = True
                    break

def compress_little(frame):
    for signal in frame.signals:
        if not signal.is_little_endian:
            return
    gap_found = True
    while gap_found:
        gap_found = False
        layout = frame.get_frame_layout()
        gap_len = None
        for byte in range(len(layout)//8):
            for bit in range(7,-1,-1):
                bit_nr = byte*8+bit
                signal_list = layout[bit_nr]
                if signal_list == []:
                    if gap_len is None:
                        gap_len = 1
                    else:
                        gap_len += 1
                else:
                    if gap_len is not None:
                        signal = layout[bit_nr][0] 
                        signal.start_bit -= gap_len
                        gap_found = True
                        break
            if gap_found:
                break

print(compress_little(test_frame))

print("\n--")
simple_show_frame(test_frame)
[a.start_bit for a in test_frame.signals]

None

--

00000000
11110000
22211111
33333322
.....333
........
........
........

[0, 12, 21, 26]